## Solutions for exercises - Improving performance

first we load the data and define functions so that we are at the same state as in the exercise notebook

In [1]:
import numpy as np
import pandas as pd

## Solutions for exercises - Improving performance

first we load the data and define functions so that we are at the same state as in the exercise notebook

In [2]:
import numpy as np
import pandas as pd
from scipy.io import loadmat

data = loadmat('../../data/microtubule_tracking/summary.mat')
data
df = pd.DataFrame()
microtubule_id = 0
for i, amppnp_concentration in enumerate(data['concentrations']):
    all_frame_to_frame_velocities = data['allSpeeds'][i][0]
    for j in range(all_frame_to_frame_velocities.shape[1]):
        df = pd.concat([df, pd.DataFrame({'AMPPNP concentration': amppnp_concentration[0], 'file name': data['fileNames'][i][0][0], 'microtubule ID': microtubule_id, 'frame-to-frame microtubule tip velocity': all_frame_to_frame_velocities[:, j]})], ignore_index=True)
        microtubule_id += 1
df = df.dropna()
controls = df[df['AMPPNP concentration']==0]
# extract data for the individual controls
control1 = controls[controls['file name'] == '2015-12-12_Run_1_AMPPNP_row_1_ATP_1_107.nd2']
control2 = controls[controls['file name'] == '2015-12-12_Run_1_AMPPNP_row_1_ATP_2_108.nd2']

def get_confidence_interval_by_bootstrapping(measurements:pd.DataFrame, bootstrap_size: int = 1000, alpha: float = 0.05):
    '''Estimate confidence intervals for data in a pandas dataframe by bootstrapping'''
    bootstrap_samples = np.empty((bootstrap_size,))
    for i in range(bootstrap_size):
        resample_index = np.random.choice(measurements.index,size=measurements.shape[0])
        bootstrap_samples[i] = np.median(measurements['frame-to-frame microtubule tip velocity'][resample_index])
    lower_bound = alpha * 100 / 2
    upper_bound = 100 - lower_bound
    return np.percentile(bootstrap_samples,[lower_bound, upper_bound])



Depending on the size of your data and the complexity of the function you use to calculate $\hat\theta$, bootstrapping can take quite some time. The computational effort required for bootsrapping was the reason, why bootstrapping was not immediately adopted by the scientific community at the time it was invented by Bradley Efron in 1979. But nowadays, with powerful computers widely available, that is not a problem any more. Considering that your experiment probably took days or even weeks, it is o.k. if the data analysis takes a few minutes or even hours. Consider it a matter of respect for your data to analyze it properly.

However, in our case, there is actually something we can do about the time the calculations are taking. Pandas is nice and all, but performance is not its strong suit. Numpy is much faster - particularly in combination with just-in-time compilation facilitated by numba. 

### Exercise 1

Let's convert our data to numpy arrays and try  `get_confidence_interval_by_bootstrapping` again.

In [3]:
numpy_control1 = control1['frame-to-frame microtubule tip velocity'].values
numpy_control2 = control2['frame-to-frame microtubule tip velocity'].values

get_confidence_interval_by_bootstrapping(numpy_control1)

AttributeError: 'numpy.ndarray' object has no attribute 'index'

So our function expects pandas tables. In order for it to work with numpy arrays, we need to rewrite it.

While we are at it, let's also make our function more modular, so we don't need to rewrite the whole thing every time we change the data type.

### Exercise 2

Let's so make our function more modular, so we don't need to rewrite the whole thing every time we change the data type.
Split the function into three parts:

In [4]:
#Turn this:
def get_confidence_interval_by_bootstrapping(measurements:pd.DataFrame, bootstrap_size: int = 1000, alpha: float = 0.05):
    '''Estimate confidence intervals for data in a pandas dataframe by bootstrapping'''
    bootstrap_samples = np.empty((bootstrap_size,))
    for i in range(bootstrap_size):
        resample_index = np.random.choice(measurements.index,size=measurements.shape[0])
        bootstrap_samples[i] = np.median(measurements['frame-to-frame microtubule tip velocity'][resample_index])
    lower_bound = alpha * 100 / 2
    upper_bound = 100 - lower_bound
    return np.percentile(bootstrap_samples,[lower_bound, upper_bound])

#into three functions:
import numba
@numba.njit #this decorator enables the numba just in time compiler for the function defined in the next line
def draw_bootstrap_sample(data: np.ndarray):
    """Draw a bootstrap sample from a 1D data set."""
    return np.random.choice(data, size=data.shape[0])

@numba.njit
def bootstrap_median(data: np.ndarray, bootstrap_size=1000):
    """Aply a function to many boostrap samples."""
    bootstrap_replicates = np.empty(bootstrap_size)
    for i in range(bootstrap_size):
        bootstrap_replicates[i] = np.median(draw_bootstrap_sample(data))
    return bootstrap_replicates

@numba.njit
def get_confidence_interval(boostrap_replicates: np.ndarray, alpha: float = 0.05):
    """Calculate the confidence interval with conficence level alpha from bootstrap replicates"""
    lower_bound = alpha * 100 / 2
    upper_bound = 100 - lower_bound
    return np.percentile(boostrap_replicates,[lower_bound, upper_bound])   


### Exercise 3:

apply your new functions to the data to calculate confidence intervals. Make sure that the results are the same as before

In [5]:
# Fixed seed for reproducibility
np.random.seed(42)

# convert pandas tables to numpy arrays
numpy_control1 = control1['frame-to-frame microtubule tip velocity'].values
numpy_control2 = control2['frame-to-frame microtubule tip velocity'].values

confidence_interval1 = get_confidence_interval(bootstrap_median(numpy_control1))
confidence_interval2 = get_confidence_interval(bootstrap_median(numpy_control2))
print('Confidence interval 1: {}\nConfidence interval 2: {}'.format(confidence_interval1, confidence_interval2))


Confidence interval 1: [774.37708725 784.96674312]
Confidence interval 2: [778.7187312  792.65464497]


### Exercise 4:

Let's benchmark our two functions:


In [6]:

print('pandas:')
%timeit get_confidence_interval_by_bootstrapping(control1)

print('numpy/numba:')
%timeit get_confidence_interval(bootstrap_median(numpy_control1))


pandas:
212 ms ± 1.34 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
numpy/numba:
47 ms ± 198 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


A factor of four - this could actually save us quite a bit of time for larger datasets or more complicated target functions.